# import

In [206]:
ratings = pd.read_csv('data/ratings.csv')
ratings_random = pd.read.csv('data/ratings_random.csv')
ratings

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3
...,...,...,...
5976474,49925,510,5
5976475,49925,528,4
5976476,49925,722,4
5976477,49925,949,5


In [118]:
ratings_train, ratings_test = train_test_split(ratings_random, test_size=0.2, random_state=322)

# surprise

In [12]:
from surprise import Reader, Dataset
from surprise.model_selection import KFold, GridSearchCV
from surprise import accuracy
from surprise.model_selection.validation import cross_validate
from surprise import NormalPredictor, KNNWithMeans, KNNWithZScore, SVD, SVDpp

/content/drive/My Drive/Packages
/content/drive/My Drive/GitHub/minor-goodbooks-recommender


In [13]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings_random, reader)

In [14]:
data_train, data_test = Dataset.load_from_df(ratings_train, reader).build_full_trainset(), Dataset.load_from_df(ratings_test, reader)

In [15]:
data_full = Dataset.load_from_df(ratings_random, reader).build_full_trainset()

In [16]:
#from surprise.model_selection.split import train_test_split as surprise_train_test_split
#data_train, data_test = surprise_train_test_split(data)

In [17]:
kf = KFold(n_splits=5, random_state=322)

In [18]:
def cross_val_score(estimator, X, scoring=None, cv=None):
    cvres_random = cross_validate(estimator, X, cv=cv, measures=[scoring])
    return cvres_random[f'test_{scoring}']

In [ ]:
alg_random = NormalPredictor()

cross_val_score(alg_random, data, cv=5, scoring='rmse').mean()

1.331807934109479

In [ ]:
start = time.time()
alg_knn = KNNWithMeans(k=5, verbose=False)

print(cross_val_score(alg_knn, data, cv=5, scoring='rmse').mean())
print(time.time()-start)

0.9174599862505092
56.99207329750061


In [ ]:
start = time.time()

alg_knn_z = KNNWithZScore(k=5, verbose=False)

print(cross_val_score(alg_knn_z, data, cv=5, scoring='rmse').mean())
print(time.time()-start)

0.9284487040641028
59.418858766555786


In [ ]:
param_grid = {'k': range(1, 10),
              'sim_options': {'name': ['msd', 'cosine', 'pearson'],
                              'user_based': [False, True]},
              'verbose' : [False]
              }

gs = GridSearchCV(KNNWithMeans, param_grid, measures=['rmse'], cv=kf)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8916482192016723
{'k': 9, 'sim_options': {'name': 'msd', 'user_based': True}, 'verbose': False}


In [ ]:
param_grid = {'k': range(1, 10),
              'sim_options': {'name': ['msd', 'cosine', 'pearson'],
                              'user_based': [False, True]},
              'verbose' : [False]
              }

gsz = GridSearchCV(KNNWithZScore, param_grid, measures=['rmse'], cv=kf)
gsz.fit(data)

print(gsz.best_score['rmse'])
print(gsz.best_params['rmse'])

0.8957300167589836
{'k': 9, 'sim_options': {'name': 'msd', 'user_based': True}, 'verbose': False}


In [19]:
param_grid_svd = {'n_factors': range(10,110,10),
                  'verbose' : [False]
              }

gssvd = GridSearchCV(SVD, param_grid_svd, measures=['rmse'], cv=kf)
gssvd.fit(data)

print(gssvd.best_score['rmse'])
print(gssvd.best_params['rmse'])

0.8682623628854061
{'n_factors': 20, 'verbose': False}


In [ ]:
param_grid_svdpp = {'n_factors': range(10,50,4),
                  'verbose' : [False]
              }

gssvdpp = GridSearchCV(SVDpp, param_grid_svdpp, measures=['rmse'], cv=kf)
gssvdpp.fit(data)

print(gssvdpp.best_score['rmse'])
print(gssvdpp.best_params['rmse'])

## recall-presision

In [ ]:
from thisproject import recommend_list

In [ ]:
knn_test = KNNWithMeans(k=9, verbose=False)
knn_reclist = recommend_list(
    ratings.loc[ratings['user_id'] == 1], 
    ratings_random, 
    knn_test, 
    verbose = True, 
    remove_rated = False
)
knn_reclist

In [ ]:
svd_test = SVD(n_factors=20, verbose=False)
svd_reclist = recommend_list(
    ratings.loc[ratings['user_id'] == 1], 
    ratings_random, 
    svd_test, 
    verbose=True, 
    remove_rated=False
)
svd_reclist

In [ ]:
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    from collections import defaultdict
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

In [ ]:
#def precision_at_k(user_id, rec_list):
    #

In [ ]:
algo = KNNWithMeans(k=9, verbose=False)

algo.fit(data_train)
predictions = algo.test(data_test)
precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=4)

print(sum(prec for prec in precisions.values()) / len(precisions))
print(sum(rec for rec in recalls.values()) / len(recalls))

In [ ]:
algo_svd = SVD(n_factors=20, verbose=False)

algo_svd.fit(data_train)
predictions = algo_svd.test(data_test)
precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4)

print(sum(prec for prec in precisions.values()) / len(precisions))
print(sum(rec for rec in recalls.values()) / len(recalls))

# spotlight

In [ ]:
import torch
import spotlight

In [ ]:
from spotlight.interactions import Interactions

data = Interactions(*[np.array(i) for _, i in ratings_random.items()])
data_train = Interactions(*[np.array(i) for _, i in ratings_train.items()])
data_test = Interactions(*[np.array(i) for _, i in ratings_test.items()])

In [ ]:
from spotlight.evaluation import rmse_score
from spotlight.factorization.explicit import ExplicitFactorizationModel

model = ExplicitFactorizationModel(loss='regression',
                                   embedding_dim=128,
                                   n_iter=10,
                                   batch_size=1024,
                                   l2=1e-9,
                                   learning_rate=1e-3,
                                   use_cuda=torch.cuda.is_available())
model.fit(data_train, verbose=True)

rmse_score(model, data_test)

Epoch 0: loss 13.886664521540695
Epoch 1: loss 3.418573094539888
Epoch 2: loss 1.1470414896379724
Epoch 3: loss 0.8211444383527077
Epoch 4: loss 0.7350682071350163
Epoch 5: loss 0.6972380533750476
Epoch 6: loss 0.6719108909496422
Epoch 7: loss 0.6488575676991705
Epoch 8: loss 0.6267038788406634
Epoch 9: loss 0.6040727829984329


0.86410011960886

In [ ]:
pred = model.predict(user_ids=58)

In [ ]:
ratings_random.loc[ratings_random['user_id'] == 58]

629          26
630          58
631          18
632          14
633           2
           ... 
5771155     279
5785098      61
5811024     702
5830792    9356
5853023    4576
Name: book_id, Length: 82, dtype: int64

In [ ]:
pred_list = pd.DataFrame(enumerate(pred), columns=['book_id', 'predcited_rating'])
pred_list.loc[
    ~pred_list['book_id'].isin(ratings_random.loc[ratings_random['user_id'] == 58].book_id)
].sort_values(ascending=False, by = 'predcited_rating').head(10)

,book_id,predcited_rating
7593,7593,5.120577
8580,8580,4.996542
5529,5529,4.977715
9004,9004,4.920152
3920,3920,4.866500
7844,7844,4.842587
9566,9566,4.834001
2151,2151,4.804973
9537,9537,4.804045
4620,4620,4.792125


# lightfm

In [5]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
import copy
import pickle
from lightfm import LightFM
from lightfm.data import Dataset

In [6]:
books = pd.read_csv('data/books_transform.csv')
book_map = pd.read_csv('data/books.csv')[['id', 'title', 'authors']]
ratings = pd.read_csv('data/ratings.csv')
item_features = sp.load_npz('data/item_features.npz')

In [222]:
dataset = Dataset()
dataset.fit(ratings_random['user_id'].unique(), np.arange(1,10001))
#dataset.fit_partial(ratings['user_id'], ratings['book_id'])

In [217]:
def itervalues(df):
    for _, i in df.iterrows():
        yield i.values

In [223]:
(interactions, weights) = dataset.build_interactions(itervalues(ratings_random))

In [224]:
weights

<2672x10000 sparse matrix of type '<class 'numpy.float32'>'
	with 298176 stored elements in COOrdinate format>

In [225]:
from lightfm.cross_validation import random_train_test_split
weights_train, weights_test = random_train_test_split(weights, test_percentage=0.2, random_state=322)

In [227]:
data = {'train' : weights_train, 
        'test': weights_test, 
        'item_features' : None, 
        'item_labels' : pd.read_csv('data/books.csv')['title'].values, 
        'item_feature_labels' : None}

In [228]:
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

model = LightFM(learning_rate=0.05, loss = 'warp')
model.fit(data['train'], epochs=10)

train_precision = precision_at_k(model, data['train'], k=10).mean()
test_precision = precision_at_k(model, data['test'], k=10, train_interactions=data['train']).mean()

train_auc = auc_score(model, data['train']).mean()
test_auc = auc_score(model, data['test'], train_interactions=data['train']).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.41, test 0.16.
AUC: train 0.93, test 0.90.


In [237]:
def evaluate(model_params, epochs=10):
    model = LightFM(**model_params, random_state=1)
    model.fit(data['train'], epochs=epochs)

    train_precision = precision_at_k(model, data['train'], k=10).mean()
    test_precision = precision_at_k(model, data['test'], k=10, train_interactions=data['train']).mean()

    train_auc = auc_score(model, data['train']).mean()
    test_auc = auc_score(model, data['test'], train_interactions=data['train']).mean()

    print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
    print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

In [258]:
for i in np.arange(10, 201, 10):
    print('N Components: ', i)
    evaluate(dict(no_components = i, learning_rate=0.05, loss='warp'))

N Components:  10
Precision: train 0.41, test 0.16.
AUC: train 0.92, test 0.90.
N Components:  20
Precision: train 0.45, test 0.18.
AUC: train 0.95, test 0.92.
N Components:  30
Precision: train 0.48, test 0.19.
AUC: train 0.96, test 0.93.
N Components:  40
Precision: train 0.50, test 0.20.
AUC: train 0.97, test 0.93.
N Components:  50
Precision: train 0.52, test 0.20.
AUC: train 0.97, test 0.93.
N Components:  60
Precision: train 0.54, test 0.20.
AUC: train 0.98, test 0.93.
N Components:  70
Precision: train 0.56, test 0.21.
AUC: train 0.98, test 0.93.
N Components:  80
Precision: train 0.58, test 0.22.
AUC: train 0.98, test 0.94.
N Components:  90
Precision: train 0.59, test 0.22.
AUC: train 0.99, test 0.94.
N Components:  100
Precision: train 0.61, test 0.22.
AUC: train 0.99, test 0.94.
N Components:  110
Precision: train 0.63, test 0.22.
AUC: train 0.99, test 0.94.
N Components:  120
Precision: train 0.64, test 0.22.
AUC: train 0.99, test 0.94.
N Components:  130
Precision: train 0

In [261]:
for i in np.arange(80, 111, 10):
    for j in np.random.exponential(0.05, 10):
        print('N Components: ', i)
        print('Learning rate:', j)
        evaluate(dict(no_components = i, learning_rate=0.05, loss='warp'))

N Components:  80
Learning rate: 0.019290148549252116
Precision: train 0.57, test 0.21.
AUC: train 0.98, test 0.94.
N Components:  80
Learning rate: 0.025272294673488207
Precision: train 0.57, test 0.22.
AUC: train 0.98, test 0.94.
N Components:  80
Learning rate: 0.018430494625011003
Precision: train 0.58, test 0.21.
AUC: train 0.98, test 0.94.
N Components:  80
Learning rate: 0.01327867393801306
Precision: train 0.58, test 0.21.
AUC: train 0.98, test 0.93.
N Components:  80
Learning rate: 0.11339931151783898
Precision: train 0.58, test 0.21.
AUC: train 0.98, test 0.94.
N Components:  80
Learning rate: 0.10547597662715243
Precision: train 0.58, test 0.21.
AUC: train 0.98, test 0.94.
N Components:  80
Learning rate: 0.15611439864262977
Precision: train 0.58, test 0.21.
AUC: train 0.98, test 0.94.
N Components:  80
Learning rate: 0.06269707542360263
Precision: train 0.58, test 0.21.
AUC: train 0.98, test 0.94.
N Components:  80
Learning rate: 0.01089150904639876
Precision: train 0.57, t

In [274]:
def evaluate_feats(model_params, epochs=10):
    model = LightFM(**model_params, random_state=1)
    model.fit(data['train'], item_features=item_features, epochs=epochs)

    train_precision = precision_at_k(model, data['train'], item_features=item_features, k=10).mean()
    test_precision = precision_at_k(model, 
                                    data['test'], 
                                    item_features=item_features, 
                                    k=10, 
                                    train_interactions=data['train']).mean()

    train_auc = auc_score(model, data['train'], item_features=item_features).mean()
    test_auc = auc_score(model, 
                         data['test'], 
                         train_interactions=data['train'], 
                         item_features=item_features).mean()

    print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
    print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

In [279]:
for i in np.arange(15):
    evaluate_feats(model_params=dict(no_components=100, loss='warp'), epochs=i)

Precision: train 0.01, test 0.00.
AUC: train 0.50, test 0.50.
Precision: train 0.40, test 0.15.
AUC: train 0.92, test 0.89.
Precision: train 0.47, test 0.18.
AUC: train 0.95, test 0.92.
Precision: train 0.50, test 0.20.
AUC: train 0.96, test 0.93.
Precision: train 0.53, test 0.21.
AUC: train 0.97, test 0.93.
Precision: train 0.55, test 0.22.
AUC: train 0.98, test 0.93.
Precision: train 0.57, test 0.23.
AUC: train 0.98, test 0.93.
Precision: train 0.59, test 0.23.
AUC: train 0.98, test 0.94.
Precision: train 0.60, test 0.24.
AUC: train 0.98, test 0.94.
Precision: train 0.61, test 0.24.
AUC: train 0.99, test 0.94.
Precision: train 0.62, test 0.24.
AUC: train 0.99, test 0.94.
Precision: train 0.62, test 0.24.
AUC: train 0.99, test 0.94.
Precision: train 0.63, test 0.24.
AUC: train 0.99, test 0.94.
Precision: train 0.64, test 0.24.
AUC: train 0.99, test 0.94.


KeyboardInterrupt: 

In [251]:
def sample_hyperparameters():
    """
    Yield possible hyperparameter choices.
    """

    while True:
        yield {
            "no_components": np.random.randint(16, 128),
            "learning_schedule": np.random.choice(["adagrad", "adadelta"]),
            #"loss": np.random.choice(["bpr", "warp", "warp-kos"]),
            "learning_rate": np.random.exponential(0.05),
            "item_alpha": np.random.exponential(1e-8),
            "user_alpha": np.random.exponential(1e-8),
            "max_sampled": np.random.randint(5, 15),
            "num_epochs": np.random.randint(5, 50),
        }


def random_search(train, test, num_samples=10, num_threads=1):
    """
    Sample random hyperparameters, fit a LightFM model, and evaluate it
    on the test set.

    Parameters
    ----------

    train: np.float32 coo_matrix of shape [n_users, n_items]
        Training data.
    test: np.float32 coo_matrix of shape [n_users, n_items]
        Test data.
    num_samples: int, optional
        Number of hyperparameter choices to evaluate.


    Returns
    -------

    generator of (auc_score, hyperparameter dict, fitted model)

    """
    import itertools
    for hyperparams in itertools.islice(sample_hyperparameters(), num_samples):
        num_epochs = hyperparams.pop("num_epochs")

        model = LightFM(**hyperparams, loss='bpr', random_state=1)
        model.fit(data['train'], epochs=num_epochs, num_threads=num_threads)

        auc = auc_score(model, data['test'], train_interactions=data['train'], num_threads=num_threads).mean()
        prec = precision_at_k(model, data['test'], k=10, train_interactions=data['train']).mean()
        
        hyperparams["num_epochs"] = num_epochs
        print(auc, prec, hyperparams)
        yield (auc, prec, hyperparams, model)

In [252]:
(auc, prec, hyperparams, model) = max(random_search(data['train'], data['test'], num_threads=1), key=lambda x: x[1])
print("Best roc_auc {}, precision {} at {}".format(auc, prec, hyperparams))

0.87021804 0.16796407 {'no_components': 68, 'learning_schedule': 'adagrad', 'learning_rate': 0.025226879613373162, 'item_alpha': 8.908310526277831e-10, 'user_alpha': 4.9360491549262306e-09, 'max_sampled': 11, 'num_epochs': 36}
0.86321425 0.13529192 {'no_components': 17, 'learning_schedule': 'adagrad', 'learning_rate': 0.12685879415614829, 'item_alpha': 1.0765226131936803e-08, 'user_alpha': 4.297429548223116e-08, 'max_sampled': 7, 'num_epochs': 7}
0.87430525 0.16021708 {'no_components': 100, 'learning_schedule': 'adadelta', 'learning_rate': 0.029186758987243763, 'item_alpha': 1.1672166047888302e-08, 'user_alpha': 1.3055982574666412e-08, 'max_sampled': 14, 'num_epochs': 34}
0.72541046 0.16208833 {'no_components': 91, 'learning_schedule': 'adagrad', 'learning_rate': 0.034357757996927245, 'item_alpha': 9.511138218904433e-09, 'user_alpha': 2.1003039203482873e-09, 'max_sampled': 8, 'num_epochs': 6}
0.61273944 0.10093563 {'no_components': 105, 'learning_schedule': 'adagrad', 'learning_rate': 

KeyboardInterrupt: 

In [366]:
model = LightFM(learning_rate=0.05, loss='warp')
model.fit(weights, epochs=10)

## setup

In [7]:
from thisproject import LightFM_Recommender

In [9]:
model = LightFM(no_components=100, learning_rate=0.05, loss='bpr', random_state=1)
lightfm = LightFM_Recommender()
lightfm.fit(ratings, model)

In [10]:
model = LightFM(no_components=100, learning_rate=0.05, loss='bpr', random_state=1)
lightfm_hybrid = LightFM_Recommender()
lightfm_hybrid.fit(ratings, model, item_feats = item_features)

In [13]:
with open('lightfm.pickle', 'wb') as f:
    pickle.dump(lightfm, f)

with open('lightfm_hybrid.pickle', 'wb') as f:
    pickle.dump(lightfm_hybrid, f)